In [1]:
%reset

from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join, isdir

database_dir = './ANFR Dataset'              #It should be made sure that there are equal number of data and ref folders here

pre_processed_dir = './Pre-processed data'

#------------The main data directory contains data and ref folder for each month's data------------#
folders_in_data_dir = [folder for folder in listdir(database_dir) if isdir(join(database_dir, folder))]
data_tables_folder = [folder for folder in folders_in_data_dir if (("ref" not in folder) and ("Ref" not in folder))]
reference_folder = [folder for folder in folders_in_data_dir if (("ref" in folder) or ("Ref" in folder))]

already_processed = [file.split('_trx')[0] for file in listdir(pre_processed_dir) if isfile(join(pre_processed_dir, file)) and "trx" in file]

print("------------------------------------ALL Folders------------------------------------")
print(folders_in_data_dir)

print("------------------------------------Data Folders------------------------------------")
print(data_tables_folder)

print("------------------------------------Reference Folders------------------------------------")
print(reference_folder)

print("------------------------------------Dates already Processed------------------------------------")
print(already_processed)

total_num_bs_df = pd.DataFrame(columns=['Dates', 'Total_NUM_BSs'])         #We will save this in a file at the end. For comparison with the number of stations of the 4 main operators.
if 'NUM_STATIONS' in already_processed:
    total_num_bs_df = pd.read_csv(join(pre_processed_dir, 'NUM_STATIONS.txt'), sep = ';')

------------------------------------ALL Folders------------------------------------
['20190529-export-etalab-data', '20150602_Tables_de_reference', '20210503-export-etalab-ref', '20170531_DATA', '20220831-export-etalab-ref', '20150829_Tables_de_reference', '20200402-export-etalab-ref', '20220228-export-etalab-ref', '20200831-export-etalab-ref', '20191001-export-etalab-data', '20221223-export-etalab-data', '20231130-export-etalab-ref', '20180831-export-etalab-ref', '20211130-export-etalab-data', '20200731-export-etalab-data', '20230301-export-etalab-ref', '20181231-export-etalab-ref', '20180930-export-etalab-data', '20160702_Tables_de_reference', '20191129-export-etalab-data', '20200430-export-etalab-ref', '20160227_DATA', '20160130_DATA', '20201030-export-etalab-ref', '20220831-export-etalab-data', '20230428-export-etalab-ref', '20151128_DATA', '20170831_Tables_de_reference', '20160430_DATA', '20230831-export-etalab-ref', '20190228-export-etalab-ref', '20170429_Tables_de_reference', '2

In [2]:
#---------------------------------Finding the mapping between the data and the ref folders---------------------------------#
dates = []
mapped_folders = []       #This list will contain each data and ref folder name in sequence. First data then ref for the same month and then so on.

months = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'aout', 'septembre', 'octobre', 'novembre', 'décembre']

for current_data_folder in data_tables_folder:
    mapped_folders.append(current_data_folder)
    
    data_date = ''
    #---------if data folder is named in this format -->'tables-supports-antennes-emetteurs-bandes-janvier-2020'---------#
    for month in months:
        if month in current_data_folder:
            month_number = months.index(month) + 1
            if month_number < 10:
                month_number = '0' + str(month_number)
            year = current_data_folder.split('-')[6]
            day = '01'
            data_date = year + month_number + day
    #---------if data folder is named in this format -->'20211029-export-etalab-data'---------Doesn't matter if there is no data in the name#
    if data_date == '':   
        if len(current_data_folder.split('-')) > 1:
            data_date = current_data_folder.split('-')[0]
        else:
            data_date = current_data_folder.split('_')[0]    #Some folders use underscore instead of dash
    dates.append(data_date)
    
    for current_ref_folder in reference_folder:
        ref_date = ''
        #---------if ref folder is named in this format -->'table-de-reference-mars-2020'---------#
        for month in months:
            if month in current_ref_folder:
                month_number = months.index(month) + 1
                if month_number < 10:
                    month_number = '0' + str(month_number)
                year = current_ref_folder.split('-')[4]
                day = '01'
                ref_date = year + month_number + day
        #---------if data folder is named in this format -->'20210801-export-etalab-ref'---------#
        if ref_date == '':
            if len(current_ref_folder.split('-')) > 1:
                ref_date = current_ref_folder.split('-')[0]
            else:
                ref_date = current_ref_folder.split('_')[0]    #Some folders use underscore instead of dash
        
        date_diff = abs(datetime.strptime(data_date, '%Y%m%d') - datetime.strptime(ref_date, '%Y%m%d'))
        
        if date_diff < timedelta(days = 7):     #Assuming that the difference between the upload of data and ref folders won't be more than 5 days. Less than 5 days mean they belong to the same set. Also assuming that different months data is uploade after atleast 5 days
            mapped_folders.append(current_ref_folder)
            break

print("----------------------Dates----------------------")
print(dates)

print("----------------------Mapped Folders (In a list one after the another)----------------------")
print(mapped_folders)

----------------------Dates----------------------
['20190529', '20170531', '20191001', '20221223', '20211130', '20200731', '20180930', '20191129', '20160227', '20160130', '20220831', '20151128', '20160430', '20161126', '20230831', '20220131', '20180228', '20230731', '20161028', '20160109', '20240131', '20180531', '20170701', '20181130', '20210801', '20231031', '20150829', '20200930', '20181231', '20180831', '20200430', '20150926', '20211029', '20150701', '20200831', '20210503', '20171222', '20230428', '20170401', '20210831', '20171031', '20231222', '20180427', '20151031', '20160827', '20211223', '20200131', '20190228', '20220228', '20170831', '20190131', '20210630', '20221130', '20160630', '20200402', '20210331', '20180330', '20170128', '20210531', '20220930', '20220531', '20161224', '20220331', '20240229', '20230531', '20201030', '20200106', '20221028', '20180629', '20181031', '20160402', '20171130', '20210129', '20150602', '20150506', '20230630', '20190628', '20220630', '20210107', '

In [3]:
#Methods for using below: For adding the FDD BW and TDD BW columns in the dataframe.
def add_FDD_BW_col(row):
    if row['DUPLEX_TYPE'] == 'F' or row['DUPLEX_TYPE'] == 'T&F':
        return row['BW']
    else:
        return 0

def add_TDD_BW_col(row):
    if row['DUPLEX_TYPE'] == 'T' or row['DUPLEX_TYPE'] == 'T&F':
        return row['BW']
    else:
        return 0

In [4]:
# Get population category of the municipality
from site_category import get_category

def add_category_col(row):
    return get_category(row['COM_CD_INSEE'])

In [5]:
def transform_height(row):
    height = row["AER_NB_ALT_BAS"]
    if not isinstance(height, str) or height == "":
        height = 30  # Default height, not an important value
    else:
        height = height.replace(",", ".")
        height = float(height)
    return height

In [6]:
encoding_type = "latin1"

def process_folder(i: int, skip_existing: bool = True):
    global total_num_bs_df
    data_path = join(database_dir, mapped_folders[i])
    reference_path = join(database_dir, mapped_folders[i + 1])

    data_date = dates[i // 2]

    if data_date in already_processed and skip_existing:
        return

    print("Currently Pre-Processing")
    print(data_path)
    print(reference_path)

    # -----------------Converting reference files to dataframes - will be used to find mapping in the data files-----------------#
    operators_df = pd.read_csv(
        join(reference_path, "SUP_EXPLOITANT.txt"),
        sep=";",
        dtype=str,
        encoding=encoding_type,
        on_bad_lines="skip",
    )  # Not entirely sure why but this encoding type works for all. On_bad_lines for ID 244 -->skipping it.
    antenna_type_df = pd.read_csv(
        join(reference_path, "SUP_TYPE_ANTENNE.txt"),
        sep=";",
        dtype=str,
        encoding=encoding_type,
    )
    # -----------------Creating dictionaries from the reference dataframes-----------------#
    operators_mapping = operators_df.set_index("ADM_ID").T.to_dict("index")
    operators_mapping = operators_mapping[
        "ADM_LB_NOM"
    ]  # The actual dictionary is a nested dictionary

    antenna_type_mapping = antenna_type_df.set_index("TAE_ID").T.to_dict("index")
    antenna_type_mapping = antenna_type_mapping[
        "TAE_LB"
    ]  # The actual dictionary is a nested dictionary

    # -----------------Reading actual datafiles as panda dataframes-----------------#
    station_df = pd.read_csv(
        join(data_path, "SUP_STATION.txt"), sep=";", dtype=str, encoding=encoding_type
    )

    # In earlier files this header is not capitalized so we need to check and get it so that we can extract this col if needed
    service_date_col_name = ""
    col_names_in_station_df = station_df.columns
    for col_name in col_names_in_station_df:
        if "dte_en_service" in col_name.lower():
            service_date_col_name = col_name

    station_df = station_df[
        ["STA_NM_ANFR", "ADM_ID", service_date_col_name]
    ]  # Selecting all the stations assuming all are in service
    # In some earlier data files, in the ADM_ID column there are commas in the data file but not in the reference file so we need to take care of this
    station_df["ADM_ID"] = station_df["ADM_ID"].str.split(",").str.get(0)
    # ----------------------Replacing the values in the data columns using the reference----------------------#
    station_df["ADM_ID"] = station_df["ADM_ID"].map(operators_mapping)

    antenna_df = pd.read_csv(
        join(data_path, "SUP_ANTENNE.txt"), sep=";", dtype=str, encoding=encoding_type
    )
    antenna_df = antenna_df[["STA_NM_ANFR", "AER_ID", "TAE_ID", "AER_NB_ALT_BAS"]]
    # ----------------------Replacing the values in the data columns using the reference----------------------#
    antenna_df["TAE_ID"] = antenna_df["TAE_ID"].map(antenna_type_mapping)
    antenna_df["AER_NB_ALT_BAS"] = antenna_df.apply(transform_height, axis=1)

    transmitter_df = pd.read_csv(
        join(data_path, "SUP_EMETTEUR.txt"), sep=";", dtype=str, encoding=encoding_type
    )

    transmitter_df = transmitter_df[
        ["STA_NM_ANFR", "EMR_ID", "EMR_LB_SYSTEME", "AER_ID"]
    ]
    # -----------------------Only cellular systems-----------------------#
    cellular_systems = ["GSM", "UMTS", "LTE", "5G"]
    transmitter_df = transmitter_df[
        transmitter_df["EMR_LB_SYSTEME"].str.split(" ").str[0].isin(cellular_systems)
    ]
    transmitter_df = transmitter_df[
        ~transmitter_df["EMR_LB_SYSTEME"].str.contains(" Expe")
    ]  # No experimential bands
    transmitter_df = transmitter_df[
        ~transmitter_df["EMR_LB_SYSTEME"].str.contains(" R")
    ]  # No GSM R
    transmitter_df = transmitter_df[
        ~transmitter_df["EMR_LB_SYSTEME"].str.contains(" P")
    ]  # No Private LTE

    band_df = pd.read_csv(
        join(data_path, "SUP_BANDE.txt"), sep=";", dtype=str, encoding=encoding_type
    )
    band_df = band_df[
        [
            "STA_NM_ANFR",
            "BAN_ID",
            "EMR_ID",
            "BAN_NB_F_DEB",
            "BAN_NB_F_FIN",
            "BAN_FG_UNITE",
        ]
    ]

    # --------------Sequential inner join operations to create one single dataframe--------------#
    station_antenna_df = station_df.merge(antenna_df, on=["STA_NM_ANFR"])
    station_antenna_transmitter_df = station_antenna_df.merge(
        transmitter_df, on=["AER_ID", "STA_NM_ANFR"]
    )
    all_df: pd.DataFrame = station_antenna_transmitter_df.merge(
        band_df, on=["EMR_ID", "STA_NM_ANFR"]
    )

    # Dropping any rows with NaN values #Especially, where DT_EN_SERVICE is null, we are assuming that the stations are not in service.
    all_df = all_df.dropna()

    # --------------Saving the total number of in service base stations--------------#
    num_bs = all_df["STA_NM_ANFR"].nunique()
    row = [data_date, num_bs]
    total_num_bs_df.loc[-1] = row
    total_num_bs_df = total_num_bs_df.reset_index(drop=True)

    # -----------------------Selecting the stations only from the main cellular operators-----------------------#
    main_operators = ["BOUYGUES TELECOM", "ORANGE", "SFR", "FREE MOBILE"]
    all_df = all_df.loc[all_df["ADM_ID"].isin(main_operators)]

    # --------------Replacing comma with decimal--------------#
    all_df["BAN_NB_F_DEB"] = all_df["BAN_NB_F_DEB"].str.replace(",", ".")
    all_df["BAN_NB_F_FIN"] = all_df["BAN_NB_F_FIN"].str.replace(",", ".")

    # --------------------Removing unnecessary info from the perspective of energy consumption calculations. Also summarizing and compressing the information for faster anlysis in the next stage--------------------#
    all_df["BW"] = all_df["BAN_NB_F_FIN"].astype(float) - all_df["BAN_NB_F_DEB"].astype(
        float
    )
    all_df["BW"] = all_df["BW"].round(1)

    all_df = all_df.drop(
        ["BAN_NB_F_FIN", "BAN_ID"], axis=1
    )  # Not dropping the date column now as we require this for power calculations

    all_df = all_df.groupby(
        [
            "STA_NM_ANFR",
            service_date_col_name,
            "ADM_ID",
            "AER_ID",
            "TAE_ID",
            "EMR_ID",
            "EMR_LB_SYSTEME",
            "BW",
            "BAN_FG_UNITE",
        ]
    ).nunique()  # Grouping the columns based on the specified columns --> This would return the number of unique values (per set) for the other columns which are not specified

    # -------The previous step returns --> Index ('STA_NM_ANFR', 'ADM_ID', 'TAE_ID', ' EMR_LB_SYSTEME', 'EMR_ID', 'BW') Columns ('BAN_NB_F_DEB'), the columns actually tell the number of unique values for each index
    # In the above dataframe if there are 2 values for starting frequency it is FDD (meaning 2 bands - one for the uplink while the other for the downlink) while 1 denotes TDD.
    # There are cases where there is a 3 in the starting frequency column.
    # This means that the transceiver implements both TDD & FDD. (2 starting frequencies for FDD while 1 for TDD)
    # This case will occur only for SFR in the 2.1 GHZ band because the BW for SFR (5.0MHZ) is same for TDD and FDD.

    # ----Hence mentioning FDD or TDD in the dataframe----#
    all_df["BAN_NB_F_DEB"] = all_df["BAN_NB_F_DEB"].map({1: "T", 2: "F", 3: "T&F"})
    all_df = all_df.rename(columns={"BAN_NB_F_DEB": "DUPLEX_TYPE"})

    # -------Creating the index back into the columns-------#
    all_df = all_df.reset_index(
        level=[
            "STA_NM_ANFR",
            service_date_col_name,
            "ADM_ID",
            "AER_ID",
            "TAE_ID",
            "EMR_LB_SYSTEME",
            "EMR_ID",
            "BW",
            "BAN_FG_UNITE",
        ]
    )

    # -------Applying the function to specify the FDD and TDD BWs with respect to each BW for each transceiver.
    all_df["FDD_BW"] = all_df.apply(add_FDD_BW_col, axis=1)
    all_df["TDD_BW"] = all_df.apply(add_TDD_BW_col, axis=1)

    all_df = all_df.drop(
        ["BW", "DUPLEX_TYPE"], axis=1
    )  # dropping the columns that we don't require anymore

    # Also note that the mentioned Bandwidth for FDD is half (only transmit or receive not combined) while for TDD, the mentioned bandwidth is the whole bandwidth

    # Getting the aggregate FDD and TDD BW for each transceiver
    all_df = all_df.groupby(
        [
            "STA_NM_ANFR",
            service_date_col_name,
            "ADM_ID",
            "AER_ID",
            "TAE_ID",
            "EMR_ID",
            "EMR_LB_SYSTEME",
            "BAN_FG_UNITE",
        ]
    )[["FDD_BW", "TDD_BW"]].sum()

    # #-------Creating the index back into the columns-------#
    all_df = all_df.reset_index(
        level=[
            "STA_NM_ANFR",
            service_date_col_name,
            "ADM_ID",
            "AER_ID",
            "TAE_ID",
            "EMR_ID",
            "EMR_LB_SYSTEME",
            "BAN_FG_UNITE",
        ]
    )

    # -----------------------------Renaming the headers to make them more clear-----------------------------#
    support_df = pd.read_csv(
        join(data_path, "SUP_SUPPORT.txt"), sep=";", dtype=str, encoding=encoding_type
    )
    support_df = support_df[["SUP_ID", "STA_NM_ANFR", "SUP_NM_HAUT", "COM_CD_INSEE"]]
    support_df['CAT'] = support_df.apply(add_category_col, axis=1)

    all_df = all_df.merge(support_df, on=["STA_NM_ANFR"])
    all_df = all_df.merge(antenna_df, on=["AER_ID", "TAE_ID", "STA_NM_ANFR"])

    all_df = all_df.rename(
        columns={
            "STA_NM_ANFR": "BS_ID",
            service_date_col_name: "DoS",
            "ADM_ID": "OPR_NAME",
            "TAE_ID": "ANT_TYPE",
            "EMR_ID": "TRX_ID",
            "EMR_LB_SYSTEME": "System",
            "BAN_FG_UNITE": "BW_UNIT",
            "AER_ID": "ANT_ID",
            "AER_NB_ALT_BAS": "ANT_HEIGHT",
            "SUP_ID": "SUP_ID",
            "SUP_NM_HAUT": "SUP_HEIGHT",
            "COM_CD_INSEE": "INSEE_CODE",
            "CAT": "CAT"  
        }
    )
    all_df = all_df.reindex(
        columns=[
            "BS_ID",
            "DoS",
            "OPR_NAME",
            "ANT_TYPE",
            "ANT_ID",
            "TRX_ID",
            "System",
            "FDD_BW",
            "TDD_BW",
            "BW_UNIT",
            "ANT_HEIGHT",
            "SUP_ID",
            "SUP_HEIGHT",
            "INSEE_CODE",
            "CAT" 
        ]
    )
    all_df = all_df.drop_duplicates(subset=["TRX_ID"])

    antenna_df = antenna_df.rename(
        columns={
            "STA_NM_ANFR": "BS_ID",
            "TAE_ID": "ANT_TYPE",
            "AER_ID": "ANT_ID",
            "AER_NB_ALT_BAS": "ANT_HEIGHT",
        }
    )
    antenna_df = antenna_df.reindex(
        columns=["BS_ID", "ANT_ID", "ANT_TYPE", "ANT_HEIGHT"]
    )

    support_df = support_df.rename(
        columns={
            "SUP_ID": "SUP_ID",
            "STA_NM_ANFR": "BS_ID",
            "SUP_NM_HAUT": "SUP_HEIGHT",
            "COM_CD_INSEE": "INSEE_CODE",
            "CAT": "CAT"
        }
    )
    support_df = support_df.reindex(
        columns=["SUP_ID", "SUP_HEIGHT", "BS_ID", "INSEE_CODE", "CAT"]
    )

    # -----------------------------Saving the dataframe to a textfile-----------------------------#
    all_df.to_csv(
        join(pre_processed_dir, data_date) + "_trx.txt",
        sep=";",
        index=False,
        encoding="utf-8",
    )
    antenna_df.to_csv(
        join(pre_processed_dir, data_date) + "_ant.txt",
        sep=";",
        index=False,
        encoding="utf-8",
    )
    support_df.to_csv(
        join(pre_processed_dir, data_date) + "_sup.txt",
        sep=";",
        index=False,
        encoding="utf-8",
    )
    print("-------------------------------------------------------------")

In [15]:
for i in range(0, len(mapped_folders), 2):
    try:
        process_folder(i, skip_existing = True)
    except Exception as e:
        print(e)
        print("Error in processing the folder")
        print(mapped_folders[i])
        print(mapped_folders[i + 1])
        continue
    
#----------------Writing the total num of stations to a textfile----------------#
total_num_bs_df.to_csv(join(pre_processed_dir, 'NUM_STATIONS.txt'), sep =';', index=False, encoding=encoding_type)

Currently Pre-Processing
./ANFR Dataset/20160227_DATA
./ANFR Dataset/20160227_Tables_de_reference
-------------------------------------------------------------
Currently Pre-Processing
./ANFR Dataset/20160130_DATA
./ANFR Dataset/20160130_Tables_de_reference
-------------------------------------------------------------
Currently Pre-Processing
./ANFR Dataset/20151128_DATA
./ANFR Dataset/20151128_Tables_de_reference
-------------------------------------------------------------
Currently Pre-Processing
./ANFR Dataset/20160430_DATA
./ANFR Dataset/20160430_Tables_de_reference
-------------------------------------------------------------
Currently Pre-Processing
./ANFR Dataset/20160109_DATA
./ANFR Dataset/20160109_Tables_de_reference
-------------------------------------------------------------
Currently Pre-Processing
./ANFR Dataset/20150829_DATA
./ANFR Dataset/20150829_Tables_de_reference
-------------------------------------------------------------
Currently Pre-Processing
./ANFR Dataset/